<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/05_agentic_ai/01_crewai_demo_agents_team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Crew ai

In [2]:
!pip install -q crewai
# Importing Crew related components
from crewai import Agent, Task, Crew

In [1]:
!pip install -q 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.5/228.5 kB 14.9 MB/s eta 0:00:00


In [3]:
# Importing CrewAI Tools
from crewai_tools import WebsiteSearchTool

In [6]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEM_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


In [21]:
from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-2.5-flash",
)

In [22]:
my_llm=gemini

In [23]:
engineering_lead = Agent(
    role = 'Engineering Lead for the engineering team, directing the work of the engineer',
    goal = '''Take the high level requirements described here and prepare a detailed design for the backend developer;
    everything should be in 1 python module; describe the function and method signatures in the module.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    Here are the requirements: {requirements}
    The module should be named {module_name} and the class should be named {class_name}''',
    backstory = '''You're a seasoned engineering lead with a knack for writing clear and concise designs.''',
    llm = my_llm,
    verbose=True,
)

In [24]:
backend_engineer = Agent(
    role = 'Python Engineer who can write code to achieve the design described by the engineering lead',
    goal = '''Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    Here are the requirements: {requirements}
    The module should be named {module_name} and the class should be named {class_name}''',
    backstory = '''You're a seasoned python engineer with a knack for writing clean, efficient code.
    You follow the design instructions carefully.
    You produce 1 python module named {module_name} that implements the design and achieves the requirements.''',
    llm = my_llm,
    verbose=True,
)

In [25]:
frontend_engineer = Agent(
    role = 'A Gradio expert to who can write a simple frontend to demonstrate a backend',
    goal = '''Write a gradio UI that demonstrates the given backend, all in one file to be in the same directory as the backend module {module_name}.
    Here are the requirements: {requirements}''',
    backstory = '''You're a seasoned python engineer highly skilled at writing simple Gradio UIs for a backend class.
    You produce a simple gradio UI that demonstrates the given backend class; you write the gradio UI in a module app.py that is in the same directory as the backend module {module_name}.''',
    llm = my_llm,
    verbose=True,
)

In [26]:
test_engineer = Agent(
    role = 'An engineer with python coding skills who can write unit tests for the given backend module {module_name}',
    goal = '''Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.''',
    backstory = '''You're a seasoned QA engineer and software developer who writes great unit tests for python code.''',
    llm = my_llm,
    verbose=True,
)

In [27]:

# Create tasks for your agents
design_task = Task(
  description="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    Here are the requirements: {requirements}. write design in markup format.""",
  agent=engineering_lead,
  verbose=True,
  output_file = "output/design.md",
  expected_output="""A python module that implements the design and achieves the requirements.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed."""

)

In [28]:
code_task = Task(
  description="""write code based on design shared by design task. code should be modular and well written""",
  agent=backend_engineer,
  expected_output="",
  context=[design_task],
  output_file= "output/{module_name}"
)

In [29]:
frontend_task = Task(
  description="""Write a gradio UI in a module app.py that demonstrates the given backend class in {module_name}.
    Assume there is only 1 user, and keep the UI very simple indeed - just a prototype or demo.
    Here are the requirements: {requirements}""",
  agent=frontend_engineer,
  expected_output="""A gradio UI in module app.py that demonstrates the given backend class.
    The file should be ready so that it can be run as-is, in the same directory as the backend module, and it should import the backend class from {module_name}.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/app.py"
)

In [30]:
test_task = Task(
  description="""Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.""",
  agent=test_engineer,
  expected_output="""A test_{module_name} module that tests the given backend module.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/test_{module_name}"
)

In [31]:

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[engineering_lead,backend_engineer,frontend_engineer,test_engineer],
  tasks=[design_task,code_task,frontend_task,test_task],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [32]:
requirements = """
A simple account management system for a trading simulation platform.
The system should allow users to create an account, deposit funds, and withdraw funds.
The system should allow users to record that they have bought or sold shares, providing a quantity.
The system should calculate the total value of the user's portfolio, and the profit or loss from the initial deposit.
The system should be able to report the holdings of the user at any point in time.
The system should be able to report the profit or loss of the user at any point in time.
The system should be able to list the transactions that the user has made over time.
The system should prevent the user from withdrawing funds that would leave them with a negative balance, or
 from buying more shares than they can afford, or selling shares that they don't have.
 The system has access to a function get_share_price(symbol) which returns the current price of a share, and includes a test implementation that returns fixed prices for AAPL, TSLA, GOOGL.
"""
module_name="accounts.py"
class_name = "Account"

template_input = {"requirements": requirements,"module_name" : module_name, "class_name" :class_name}

In [ ]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=template_input)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engineering Lead for the engineering team, directing the work of the engineer                           │
│                                                                                                                 │
│  Task: Write a python module that implements the design described by the engineering lead, in order to achieve  │
│  the requirements.                                                                                              │
│      Here are the requirements:                                                                                 │
│  A simple account management system for a trading simulation platform.                                          │
│  The system should allow users to create an account, deposit funds, and withdraw funds.                         │
│  The system should allow users to record that they have bought or sold shares, providing a quantity.            │
│  The system should calculate the total value of the user's portfolio, and the profit or loss from the initial   │
│  deposit.                                                                                                       │
│  The system should be able to report the holdings of the user at any point in time.                             │
│  The system should be able to report the profit or loss of the user at any point in time.                       │
│  The system should be able to list the transactions that the user has made over time.                           │
│  The system should prevent the user from withdrawing funds that would leave them with a negative balance, or    │
│   from buying more shares than they can afford, or selling shares that they don't have.                         │
│   The system has access to a function get_share_price(symbol) which returns the current price of a share, and   │
│  includes a test implementation that returns fixed prices for AAPL, TSLA, GOOGL.                                │
│  . write design in markup format.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engineering Lead for the engineering team, directing the work of the engineer                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thought: I have received the detailed requirements and the expected output format. I need to act as an         │
│  Engineering Lead and provide a clear, concise design for a Python module named `accounts.py` that contains an  │
│  `Account` class. This design will detail function and method signatures, internal state, and logic to satisfy  │
│  all specified requirements, including a self-contained test implementation for `get_share_price`.              │
│                                                                                                                 │
│  Here's my plan to construct the design:                                                                        │
│                                                                                                                 │
│  1.  **Module Header**: Define the module name and overall purpose.                                             │
│  2.  **External Dependencies/Helpers**: Define the `get_share_price` function with its test implementation.     │
│  3.  **Account Class Definition**:                                                                              │
│      *   **Class Docstring**: Explain the purpose of the `Account` class.                                       │
│      *   **Internal State Variables**: Describe `_balance`, `_holdings`, `_transactions`, and                   │
│  `_initial_deposit`.                                                                                            │
│      *   **`__init__` method**: Signature and logic for account initialization.                                 │
│      *   **`_record_transaction` private helper**: Signature and logic for logging all operations.              │
│      *   **Public Methods**: For each requirement, define a method with its signature, parameters, return       │
│  type, and detailed logic, including validation checks.                                                         │
│          *   `deposit(amount)`                                                                                  │
│          *   `withdraw(amount)`                                                                                 │
│          *   `buy_shares(symbol, quantity)`                                                                     │
│          *   `sell_shares(symbol, quantity)`                                                                    │
│          *   `get_balance()`                                                                                    │
│          *   `get_holdings()`                                                                                   │
│          *   `get_portfolio_value()`                                                                            │
│          *   `get_profit_loss()`                                                                                │
│          *   `get_transactions()`                                                                               │
│  4.  **Error Handling**: Specify how invalid operations are handled (returning `bool` for simplicity as per     │
│  "simple system").                                                                                              │
│  5.  **Self-Containment**: Ensure all necessary imports and the helper function are part of the module design.  │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Python Engineer who can write code to achieve the design described by the engineering lead              │
│                                                                                                                 │
│  Task: write code based on design shared by design task. code should be modular and well written                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Python Engineer who can write code to achieve the design described by the engineering lead              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```python                                                                                                      │
│  import datetime                                                                                                │
│  from typing import Dict, List, Any, Optional                                                                   │
│                                                                                                                 │
│  # --- External Helper Function (Mock Implementation for Testing) ---                                           │
│  def get_share_price(symbol: str) -> Optional[float]:                                                           │
│      """                                                                                                        │
│      Simulates fetching the current market price for a given share symbol.                                      │
│      Returns a fixed price for common symbols, or None if not found.                                            │
│      """                                                                                                        │
│      prices = {                                                                                                 │
│          "AAPL": 170.00,                                                                                        │
│          "TSLA": 250.00,                                                                                        │
│          "GOOGL": 1500.00,                                                                                      │
│          "MSFT": 400.00,                                                                                        │
│          "AMZN": 180.00,                                                                                        │
│      }                                                                                                          │
│      return prices.get(symbol.upper())                                                                          │
│                                                                                                                 │
│  # --- End External Helper Function ---                                                                         │
│                                                                                                                 │
│                                                                                                                 │
│  class Account:                                                                                                 │
│      """                                                                                                        │
│      Manages a user's trading account, including cash balance, share holdings,                                  │
│      and transaction history for a trading simulation platform.                                                 │
│      """                                                                                                        │
│                                                                                                                 │
│      # --- Internal State Variables ---                                                                         │
│      _initial_deposit: float                           

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: A Gradio expert to who can write a simple frontend to demonstrate a backend                             │
│                                                                                                                 │
│  Task: Write a gradio UI in a module app.py that demonstrates the given backend class in accounts.py.           │
│      Assume there is only 1 user, and keep the UI very simple indeed - just a prototype or demo.                │
│      Here are the requirements:                                                                                 │
│  A simple account management system for a trading simulation platform.                                          │
│  The system should allow users to create an account, deposit funds, and withdraw funds.                         │
│  The system should allow users to record that they have bought or sold shares, providing a quantity.            │
│  The system should calculate the total value of the user's portfolio, and the profit or loss from the initial   │
│  deposit.                                                                                                       │
│  The system should be able to report the holdings of the user at any point in time.                             │
│  The system should be able to report the profit or loss of the user at any point in time.                       │
│  The system should be able to list the transactions that the user has made over time.                           │
│  The system should prevent the user from withdrawing funds that would leave them with a negative balance, or    │
│   from buying more shares than they can afford, or selling shares that they don't have.                         │
│   The system has access to a function get_share_price(symbol) which returns the current price of a share, and   │
│  includes a test implementation that returns fixed prices for AAPL, TSLA, GOOGL.                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯